# Creating a csv file of Finger Lakes Region Wineries scraped from the web
content grabbed from www.americanwineryguide.com

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

In [229]:
result = requests.get("http://www.americanwineryguide.com/regions/wineries_list/finger-lakes-ava/")

In [230]:
src = result.content
soup = BeautifulSoup(src,"lxml")

In [231]:
#initialize some lists
urls = []
winery_list_urls = []
phone_p = []

#### the source website has everything referenced from one page with a bunch of links in html lists; the following creates a list of all the sub-links for each winery listed

In [232]:
for wineries in soup.find_all("li"):
    a_tag = wineries.find('a')
    urls.append(a_tag.attrs['href'])

#### get rid of the links that don't refer to a winery

In [233]:
for x in range(0, len(urls)):
    url_path = urls[x].split("/")
    if len(url_path) > 1:
        if url_path[1] == "wineries":
            winery_list_urls.append(urls[x])
    

#### create our csv file and add column headings

In [234]:
csv_file = open('ny_wineries.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['name', 'address_street', 'address_town','address_state', 
                     'address_zip', 'phone', 'email_url', 'winery_url', 'maps_url'])


96

#### loop through the list of sub-urls to get the details for each winery write it out as a row in our csv file

In [235]:
for url in winery_list_urls:
    request_url = f"http://www.americanwineryguide.com{url}" 
    result = requests.get(request_url)
    src = result.content
    soup = BeautifulSoup(src,"lxml")
    winery = soup.find('div', id="winery_detail_box1a")

    #Get the winery name (skip it if no name found)    
    try:
        name = winery.b.text.split('\n')[0]
    except Exception as e:
        break
    #Get the winery's website url
    try:
        winery_url = winery.find('a').attrs['href']
    except Exception as e:
        winery_url = None
    #Get the Google Maps link for this winery
    try:
        maps_link = winery.select('a[href^="http://maps.google.com/"]')
        maps_link[0] = str(maps_link[0])
        maps_url = maps_link[0].split('\"')[1]
    except Exception as e:
        maps_url = None
    #Get the street address and break it out into seperate fields    
    try:
        address = maps_link[0].split('\"')[-1].split('\n')
        address_street = address[1]
        address_town = address[-2].split(',')[0]
        address_state = "NY"
        address_zip = address[-2].split(' ')[-1]
    except Exception as e:
        address_street = None
        address_town = None
        address_state = "NY"
        address_zip = None
    #Get the winery's phone number    
    try:
        phone_p = str(winery.a.find_next('p'))
        phone = phone_p.split(' ')[-1].split('<')[0]
    except Exception as e:
        phone = None
    #Get the winery's email address
    try:
        email_link = winery.select('a[href^="mailto:"]')
        email_link[0] = str(email_link[0])
        email_url = email_link[0].split('\"')[1]
    except Exception as e:
        email_url = None
    #write all of the info for this winery to the csv file
    csv_writer.writerow([name, address_street, address_town, address_state, 
                         address_zip, phone, email_url, winery_url, maps_url])
csv_file.close()

## Testing down below...

In [213]:
url = "/wineries/americana-vineyards-winery/"
request_url = f"http://www.americanwineryguide.com{url}" 
result = requests.get(request_url)
src = result.content
soup = BeautifulSoup(src,"lxml")
winery = soup.find('div', id="winery_detail_box1a")